In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv() # OPENAI_API_KEY

llm = ChatOpenAI(model="gpt-4o")

def add(a:int, b:int) -> int:
    """Function that take two integers a and b and return the sum of them.
    Args:
        a: The first integer.
        b: The second integer.
        
    Returns:
        The sum of a and b.
    """
    return a - b

TOOLS = [add]

llm_with_tools = llm.bind_tools(TOOLS)

from langgraph.graph.message import add_messages
from typing import TypedDict, List, Annotated
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage

class AgentState(TypedDict):
    messages: Annotated[List, add_messages]
    # messages: List[BaseMessage]

from langchain_core.messages import SystemMessage

def assistant(state: AgentState):

    sys_msg = SystemMessage(
        content="You are a helpful assistant that can call tools to answer questions."
    )

    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

In [2]:
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

memory_saver = MemorySaver()

graph_builder = StateGraph(AgentState)

graph_builder.add_node("assistant", assistant)
graph_builder.add_node("tools", ToolNode(TOOLS))

graph_builder.add_edge(START, "assistant")
graph_builder.add_conditional_edges(
    "assistant",
    tools_condition,
    {
        "tools": "tools",
        "__end__": END
    }
)
graph_builder.add_edge("tools", "assistant")

agent_graph = graph_builder.compile(checkpointer=memory_saver)

In [5]:
from langchain_core.messages import HumanMessage

agent_input: AgentState = {
    "messages": [
        HumanMessage(content="What was the first question i asked?")
    ]
}

response = agent_graph.invoke(agent_input, config={"configurable": {"thread_id": "1"}})

for message in response["messages"]:
    print("====\nMessage type:")
    print(message.type)
    print("Message content:")
    print(message)

====
Message type:
human
Message content:
content='What is the sum of 2 and 2?' additional_kwargs={} response_metadata={} id='b18197ee-995e-4487-bd43-28d3b2bb3b08'
====
Message type:
ai
Message content:
content='' additional_kwargs={'tool_calls': [{'id': 'call_XswGXAgONURZqICzVK9z5RS9', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-Bi7dJss8YfL4OEmSQ85leN9C452nX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--a61003ce-8b0b-4daf-8c19-2d8f052b6290-0' tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id':

In [4]:
print(response)

{'messages': [HumanMessage(content='What is the sum of 2 and 2?', additional_kwargs={}, response_metadata={}, id='b18197ee-995e-4487-bd43-28d3b2bb3b08'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XswGXAgONURZqICzVK9z5RS9', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-Bi7dJss8YfL4OEmSQ85leN9C452nX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a61003ce-8b0b-4daf-8c19-2d8f052b6290-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'call_XswGXAgONURZqICzVK9z5RS9', '